### Training of modified U-Net for Optic Disc on RIM-ONE v3 database (cross-validation fold #0).

In [ ]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import matplotlib.pyplot as plt

from model_rimone import *
from process_images_rimone import *
from process_result import *

import h5py
import os

print('Keras version:', keras.__version__)
print('TensorFlow version:', tf.__version__)

In [ ]:
suf = '_gap_20'
epochs = NUM_EPOCHS
spe = SPE
img_size = IMG_SIZE
arq_name = 'drishti_rim_one_drions_rgb_blue'
lr_cup = 7e-4
prefix_results = 'rgb_blue'

arch_name_cup = "Cup, log_dice loss"
arch_name_disc = "Disc, log_dice loss"

In [ ]:
rim_one_images, rim_one_cups, rim_one_discs, rim_one_labels = get_images_and_labels(suf)

In [ ]:
h5f = h5py.File(os.path.join(os.path.dirname(os.getcwd()), 'data', 'hdf5_datasets', 'DRISHTI_GS.hdf5'), 'r')
suf = '_20'
#-----------------------------------------------------------------------------------------------------------#
drishti_img_test =  list(h5f['DRISHTI-GS-test/512 px/img_cropped' + suf])
drishti_cup_test =  list(h5f['DRISHTI-GS-test/512 px/cup_cropped' + suf])
drishti_disc_test =  list(h5f['DRISHTI-GS-test/512 px/disc_cropped' + suf])

drishti_img_train =  list(h5f['DRISHTI-GS/512 px/img_cropped' + suf])
drishti_cup_train =  list(h5f['DRISHTI-GS/512 px/cup_cropped' + suf])
drishti_disc_train = list(h5f['DRISHTI-GS/512 px/disc_cropped' + suf])

In [ ]:
h5f = h5py.File(os.path.join(os.path.dirname(os.getcwd()), 'data', 'hdf5_datasets', 'DRIONS-DB.hdf5'), 'r')
drions_img = list(h5f['DRIONS-DB/512 px/img_cropped_gap_20'])
drions_disc = list(h5f['DRIONS-DB/512 px/disc_cropped_gap_20'])

In [ ]:
num_splits = 5
train_idx_cv, test_idx_cv = [], []
for _train_idx, _test_idx in KFold(n_splits=num_splits, random_state=1, shuffle=True).split(rim_one_images):
    train_idx_cv.append(_train_idx)
    test_idx_cv.append(_test_idx)
    
train_idx = train_idx_cv[0]
test_idx = test_idx_cv[0]
print(len(train_idx))
print(len(test_idx))

In [ ]:
for i in range(len(drishti_img_train)):
    drishti_cup_train[i] = drishti_cup_train[i][..., None]
    drishti_disc_train[i] = drishti_disc_train[i][..., None]

for i in range(len(drishti_img_test)):
    drishti_cup_test[i] = drishti_cup_test[i][..., None]
    drishti_disc_test[i] = drishti_disc_test[i][..., None]


In [ ]:
def join_images(train_idx, images, cups, discs):
    for idx in range(len(train_idx)):
        i = train_idx[idx]
        
        images.append(rim_one_images[i])
        cups.append(rim_one_cups[i])
        discs.append(rim_one_discs[i])
        
def create_labels_list(test_idx, origin_labels):
    labels = []
    
    for i in range(51):
        labels.append(0)
    
    for idx in range(len(test_idx)):
        i = test_idx[idx]
        labels.append(origin_labels[i])
    
    return labels

In [ ]:
for k in range(num_splits):
    folder_name = root_path_kfold(arq_name, k)
    
    img_train = drishti_img_train.copy()
    cup_train = drishti_cup_train.copy()
    disc_train = drishti_disc_train.copy()
    
    img_test = drishti_img_test.copy()
    cup_test = drishti_cup_test.copy()
    disc_test = drishti_disc_test.copy()

    join_images(train_idx_cv[k], img_train, cup_train, disc_train)
    join_images(test_idx_cv[k], img_test, cup_test, disc_test)
    
    
    img_train_blue = get_color_channel(2, img_train)
    
    train_idx = np.arange(0, len(train_idx_cv[k]) + 50)
    test_idx  = np.arange(0, len(test_idx_cv[k]) + 51)
    
    labels = create_labels_list(test_idx_cv[k], rim_one_labels)
    
    print('------------- training cup --------------')
    
    X_valid, Y_valid = next(data_generator(img_test, cup_test, train_idx, test_idx, train_or_test = 'test', batch_size=100, stationary=True))
    model = create_compile_model(img_size, lr_cup)
    weights_folder = os.path.join(folder_name, 'models_weights', '{},{}'.format(datetime.now().strftime('%d.%m,%H-%M'), arch_name_cup))
    
    history_cup = train_cup(img_train_blue, cup_train, 'cup_' + prefix_results, 
                         model, epochs, X_valid, Y_valid, img_size, spe, train_idx, test_idx, weights_folder)
    
    pred_iou, pred_dice, pred_cup = predict(img_test, X_valid, Y_valid, model, img_size, test_idx)
    save_iou_dice(np.mean(pred_iou), np.mean(pred_dice), folder_name, prefix_results + '_mean_iou_dice_cup.csv')
    save_predict_images(folder_name, 'cups', test_idx, pred_cup)
        
    print('------------- training disc --------------')
    
    X_valid, Z_valid = next(data_generator(img_test, disc_test, train_idx, test_idx, train_or_test='test', batch_size=100, stationary=True))
    model_disc = create_compile_model(img_size, 1e-3)
    weights_folder = os.path.join(folder_name, 'models_weights', '{},{}'.format(datetime.now().strftime('%d.%m,%H-%M'), arch_name_disc))
    
    img_train.extend(drions_img)
    disc_train.extend(drions_disc)
    
    img_train_blue = get_color_channel(2, img_train)
    
    train_idx = np.arange(0, len(train_idx_cv[k]) + 110 + 50)

    history_disc = train_disc(img_train_blue, disc_train, 'disc_' + prefix_results, 
                         model_disc, epochs, X_valid, Z_valid, img_size, spe, train_idx, test_idx, weights_folder)
    
    pred_iou, pred_dice, pred_disc = predict(img_test, X_valid, Z_valid, model_disc, img_size, test_idx)
    save_iou_dice(np.mean(pred_iou), np.mean(pred_dice), folder_name, prefix_results + '_mean_iou_dice_disc.csv')
    save_predict_images(folder_name, 'discs', test_idx, pred_disc)
    
    print('------------- results --------------')
    
    plot_results(history_cup, epochs, folder_name, arq_name + '_cup')
    plot_results(history_disc, epochs, folder_name, arq_name + '_disc')
    df, diametros = create_table_result_rimone(pred_cup, pred_disc, test_idx, folder_name, prefix_results, labels) 
        
    img_train.clear()
    cup_train.clear()
    disc_train.clear()
    
    img_test.clear()
    cup_test.clear()
    disc_test.clear()
   